In [1]:
# basic (built-in) Python packages
import numpy as np
from numpy.linalg import norm
import pandas as pd
import matplotlib.pyplot as plt

# advanced (built-in) Python packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize

# my implemented Python functions and classes
from data.generate_data import generate_data
from model.BaseModel import BaseModel
from model.Initial import Initial
from model.OS import OS
from model.MS import MS
from model.ORACLE_beta import ORACLE_beta
from model.ORACLE_sigma import ORACLE_sigma

In [2]:
def compute_rmse(est, true):
    est = est.ravel()
    true = true.ravel()
    assert est.shape == true.shape
    length = len(est)
    rmse = norm(est - true) / np.sqrt(length)
    return rmse

# Hyperparameters

In [3]:
seed = 0
np.random.seed(seed=seed)

N = 5000        # the size of the unlabeled dataset
r = 0.5
n = int(N * r)            # pilot sample size
alpha = 1 # n**(-0.1)  
print(f"alpha={alpha:.4f}")

p = 20          # feature dimension
K = 2           # (K+1) classes
M = int(0.5 * int(n / np.log(n)))          # the size of the annotator pool #int(n / np.log(n))
print(f"[n*alpha={int(n*alpha)}/N={N}]")
print(f"[M={M}] vs [n*alpha={int(n*alpha)}]")

alpha=1.0000
[n*alpha=2500/N=5000]
[M=159] vs [n*alpha=2500]


# Data Generation

In [4]:
beta, sigma, theta, X, Y, X1, X2, Y1, Y2, A1, AY1 = generate_data(K, p, N, n, M, alpha, seed=0)

True Labels 2    1866
1    1807
0    1327
dtype: int64 



In [5]:
sigma

array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.

# Initial Estimator

In [6]:
init_model = Initial(X1, AY1, A1, K)
init_beta, init_sigma, init_betams = init_model.init_param()

In [7]:
init_beta_rmse = compute_rmse(init_beta, beta[1:])
init_sigma_rmse = compute_rmse(init_sigma, sigma)
print(f"Init beta:  {init_beta_rmse:.7f}")
print(f"Init sigma: {init_sigma_rmse:.7f}")

Init beta:  0.0042739
Init sigma: 0.3448665


# One Step / Two Step / Multiple Step

- OS:

In [11]:
os_model = OS(X1, AY1, A1, K, init_beta, init_sigma)
os_beta, os_sigma = os_model.update_alg(max_steps=1, tol=1e-5)

os_beta_rmse = compute_rmse(os_beta, beta[1:])
os_sigma_rmse = compute_rmse(os_sigma, sigma)
print(f"\nOS beta: {os_beta_rmse:.7f}")
print(f"OS sigma: {os_sigma_rmse:.7f}")

######## [Step 1] ########
norm(gradient): 10.3296093

OS beta: 0.0011264
OS sigma: 0.1798813


In [12]:
os_beta_rmse = compute_rmse(os_beta, beta[1:])
os_sigma_rmse = compute_rmse(os_sigma, sigma)
print(f"OS beta: {os_beta_rmse:.7f}")
print(f"OS sigma: {os_sigma_rmse:.7f}")

OS beta: 0.0011264
OS sigma: 0.1798813


- TS:

In [13]:
ts_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
ts_beta, ts_sigma = ts_model.update_alg(max_steps=2, tol=1e-5)

######## [Step 1] ########
norm(gradient): 10.3296093
######## [Step 2] ########
norm(gradient): 0.8735038


In [14]:
ts_beta_rmse = compute_rmse(ts_beta, beta[1:])
ts_sigma_rmse = compute_rmse(ts_sigma, sigma)
print(f"TS beta: {ts_beta_rmse:.7f}")
print(f"TS sigma: {ts_sigma_rmse:.7f}")

TS beta: 0.0011310
TS sigma: 0.1798813


- MS:

In [15]:
ms_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
ms_beta, ms_sigma = ms_model.update_alg(max_steps=3, tol=1e-5, true_beta=beta[1:].ravel(), echo=False)

In [16]:
ms_beta_rmse = compute_rmse(ms_beta, beta[1:])
ms_sigma_rmse = compute_rmse(ms_sigma, sigma)
print(f"MS beta:  {ms_beta_rmse:.7f}")
print(f"MS sigma: {ms_sigma_rmse:.7f}")

MS beta:  0.0011312
MS sigma: 0.1798813


# Oracle_beta

In [17]:
orab_model = ORACLE_beta(X1, AY1, A1, K, init_beta, sigma)
oracle_beta = orab_model.update_alg(max_steps=5, tol=1e-5, true_beta=beta[1:])

oracle_beta_rmse = compute_rmse(oracle_beta, beta[1:])
print(f"Oracle beta: {oracle_beta_rmse:.7f}")

######## [Step 1] ########
norm(gradient): 9.8428303
RMSE(beta): 0.0064970
######## [Step 2] ########
norm(gradient): 0.2226651
RMSE(beta): 0.0063049
######## [Step 3] ########
norm(gradient): 0.0006608
RMSE(beta): 0.0063064
Oracle beta: 0.0009880


In [18]:
# diff_mom, diff_son = orabeta_model.check(beta[1:], sigma)
# plt.boxplot([diff_mom, diff_son.ravel()])
# plt.show()

# Oracle_sigma

In [ ]:
oras_model = ORACLE_sigma(X1, AY1, A1, K, init_beta, init_sigma)
oracle_sigma = oras_model.update_alg(max_steps=10, tol=1e-5, true_sigma=sigma)

######## [Step 1] ########
norm(sigma): 0.3037004
######## [Step 2] ########
norm(sigma): 0.0331025
######## [Step 3] ########
norm(sigma): 0.0034648


In [ ]:
oracle_sigma_rmse = compute_rmse(oracle_sigma, sigma)
print(f"Oracle sigma: {oracle_sigma_rmse:.7f}")

# Results

In [ ]:
print(f"Init beta: {init_beta_rmse:.7f}")
print(f"OS   beta: {os_beta_rmse:.7f}")
print(f"TS   beta: {ts_beta_rmse:.7f}")
print(f"MS   beta: {ms_beta_rmse:.7f}")
print(f"Ora  beta: {oracle_beta_rmse:.7f}")

In [ ]:
print(f"Init sigma: {init_sigma_rmse:.7f}")
print(f"OS   sigma: {os_sigma_rmse:.7f}")
print(f"TS   sigma: {ts_sigma_rmse:.7f}")
print(f"MS   sigma: {ms_sigma_rmse:.7f}")
print(f"Ora  sigma: {oracle_sigma_rmse:.7f}")

# Compare Experiments

In [3]:
p = 20                           # feature dimension
K = 2                            # (K+1) classes
r = 0.5

In [4]:
B = 1000
RMSE_results = []
NAME_results = []

In [5]:
alpha_list = [0.25, 0.75]
N_list = [2000, 5000, 10000,]

In [ ]:
for N in N_list:
    n = int(N * r)                 # pilot sample size
    M = int(0.5 * n / np.log(n))   # the size of the annotator pool
    
    for alpha in alpha_list:
        print(f"[n*alpha={int(n*alpha)}/N={N}]")
        print(f"[M={M}] vs [n*alpha={int(n*alpha)}]")
        
        for seed in range(0, int(B/4)):
            RMSE_list = [seed, n, alpha, M, K]
            NAME_list = ["seed", "n", "alpha", "M", "K"]
            np.random.seed(seed)
            beta, sigma, theta, X, Y, X1, X2, Y1, Y2, A1, AY1 = generate_data(K, p, N, n, M, alpha, seed=seed)
        
            # Initial Estimator
            print(f"\n########## Initial Estimator ##########")
            init_model = Initial(X1, AY1, A1, K)
            init_beta, init_sigma, init_betams = init_model.init_param()
            init_beta_rmse = compute_rmse(init_beta, beta[1:])
            init_sigma_rmse = compute_rmse(init_sigma, sigma)
            RMSE_list += [init_beta_rmse, init_sigma_rmse]
            NAME_list += ["init_beta", "init_sigma"]
            
            # OS (One-Step) Estimator 
            print(f"\n########## OS Estimator ##########")
            os_model = OS(X1, AY1, A1, K, init_beta, init_sigma)
            os_beta, os_sigma = os_model.update_alg(max_steps=1, tol=1e-5, true_beta=beta[1:].ravel())
            os_beta_rmse = compute_rmse(os_beta, beta[1:])
            os_sigma_rmse = compute_rmse(os_sigma, sigma)
            RMSE_list += [os_beta_rmse, os_sigma_rmse]
            NAME_list += ["os_beta", "os_sigma"]
        
            # TS (Two-Step) Estimator 
            print(f"\n########## TS Estimator ##########")
            ts_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
            ts_beta, ts_sigma = ts_model.update_alg(max_steps=2, tol=1e-5, true_beta=beta[1:].ravel())
            ts_beta_rmse = compute_rmse(ts_beta, beta[1:])
            ts_sigma_rmse = compute_rmse(ts_sigma, sigma)
            RMSE_list += [ts_beta_rmse, ts_sigma_rmse]
            NAME_list += ["ts_beta", "ts_sigma"]
        
            # MS (Multiple-Step) Estimator 
            print(f"\n########## MS Estimator ##########")
            ms_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
            ms_beta, ms_sigma = ms_model.update_alg(max_steps=3, tol=1e-5, true_beta=beta[1:].ravel())
            ms_beta_rmse = compute_rmse(ms_beta, beta[1:])
            ms_sigma_rmse = compute_rmse(ms_sigma, sigma)
            RMSE_list += [ms_beta_rmse, ms_sigma_rmse]
            NAME_list += ["ms_beta", "ms_sigma"]
        
            # Oracle_beta
            print(f"\n########## ORACLE_beta Estimator ##########")
            oracle_model = ORACLE_beta(X1, AY1, A1, K, init_beta, sigma)
            oracle_beta = oracle_model.update_alg(max_steps=10, tol=1e-5, true_beta=beta[1:])
            oracle_beta_rmse = compute_rmse(oracle_beta, beta[1:])
            
            # Oracle_sigma
            print(f"\n########## ORACLE_sigma Estimator ##########")
            oracle_model = ORACLE_sigma(X1, AY1, A1, K, beta[1:], init_sigma)
            oracle_sigma = oracle_model.update_alg(max_steps=10, tol=1e-5, true_sigma=sigma)
            oracle_sigma_rmse = compute_rmse(oracle_sigma, sigma)
            RMSE_list += [oracle_beta_rmse, oracle_sigma_rmse]
            NAME_list += ["oracle_beta", "oracle_sigma"]
        
            # Print Results
            print()
            print(f"Init beta: {init_beta_rmse:.7f}")
            print(f"Ora  beta: {oracle_beta_rmse:.7f}")
            print(f"OS   beta: {os_beta_rmse:.7f}")
            print(f"TS   beta: {ts_beta_rmse:.7f}")
            print(f"MS   beta: {ms_beta_rmse:.7f}")
            print()
            print(f"Init sigma: {init_sigma_rmse:.7f}")
            print(f"Ora  sigma: {oracle_sigma_rmse:.7f}")
            print(f"OS   sigma: {os_sigma_rmse:.7f}")
            print(f"TS   sigma: {ts_sigma_rmse:.7f}")
            print(f"MS   sigma: {ms_sigma_rmse:.7f}")
        
            # Record Results
            RMSE_results.append(RMSE_list)
            a = pd.DataFrame(RMSE_results, columns=NAME_list)
            a.to_csv(f"./results/[K={K}]rmse_data3.csv")

[n*alpha=500/N=2000]
[M=72] vs [n*alpha=500]
True Labels 2    743
1    723
0    534
dtype: int64 


########## Initial Estimator ##########

########## OS Estimator ##########
######## [Step 1] ########
norm(gradient): 3.3494394
RMSE(beta): 0.0553144

########## TS Estimator ##########
######## [Step 1] ########
norm(gradient): 3.3494394
RMSE(beta): 0.0553144
######## [Step 2] ########
norm(gradient): 0.4568547
RMSE(beta): 0.0419953

########## MS Estimator ##########
######## [Step 1] ########
norm(gradient): 3.3494394
RMSE(beta): 0.0553144
######## [Step 2] ########
norm(gradient): 0.4568547
RMSE(beta): 0.0419953
######## [Step 3] ########
norm(gradient): 0.4127976
RMSE(beta): 0.0423110

########## ORACLE_beta Estimator ##########
######## [Step 1] ########
norm(gradient): 1.6895135
RMSE(beta): 0.0430921
######## [Step 2] ########
norm(gradient): 0.0375582
RMSE(beta): 0.0426521
######## [Step 3] ########
norm(gradient): 0.0000696
RMSE(beta): 0.0426507

########## ORACLE_sigma Estimat

In [ ]:
print()